In [ ]:
import nilearn
from nilearn import image
from nilearn import glm
from nilearn import plotting
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import os

In [ ]:
time_repetition = 2
beta=[]

In [ ]:
templates = dict(
    func = os.path.join("derivatives","preprocessed","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"),
    confound = os.path.join("derivatives","preprocessed","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_desc-confounds_timeseries.tsv"),
    event = os.path.join("BIDSdata","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_events.tsv")
)

In [ ]:
# set all the nan as mean value in confound variables
def replace_nan(regressor_values):
    # calculate the mean value of the regressor:
    mean_value = regressor_values.mean(skipna=True)
    # replace all values containing nan with the mean value:
    regressor_values[regressor_values.isnull()] = mean_value
    # return list of the regressor values:
    return list(regressor_values)

In [ ]:
sublist = [
    "06","07","08","09","10","11",
    "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33",
    "36","37","38","39",
    "41","42","43","44"
    ]
runlist = [
    "01","02","03","04","05"
    ]

In [ ]:
for subid in sublist:
    for runid in runlist:
        # 由于有的被试的run是残缺的，需要跳过这些不存在的run，以免报错
        if not os.path.exists(templates["func"].format(subid=subid,runid=runid)):
            print(f"sub-{subid} run-{runid} does not exist, skip")
            continue
        else:
            print(f"sub-{subid} run-{runid} exists")
            print(f"sub-{subid} run-{runid} processing")
            func = templates["func"].format(subid=subid,runid=runid)
            confound = templates["confound"].format(subid=subid,runid=runid)
            event = templates["event"].format(subid=subid,runid=runid)
            # Load the data
            confound_data = pd.read_csv(confound, sep="\t")
            event_data = pd.read_csv(event, sep="\t")
            func_run = image.load_img(func)
            n_scans = func_run.shape[-1]
            # confounds, including head motion, white matter, and CSF
            confound_names = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'csf', 'white_matter']
            regressors = [replace_nan(confound_data[conf]) for conf in confound_names]
            regressors = np.array(regressors).T
            frame_times = np.arange(n_scans) * time_repetition

            # 设计矩阵
            design_matrix = glm.first_level.make_first_level_design_matrix(
                events = event_data,
                frame_times = frame_times,
                drift_model = "polynomial",
                add_regs = regressors,
                add_reg_names= confound_names,
                high_pass = 1/128,
                oversampling=100)
            plotting.plot_design_matrix(
                design_matrix
            )
            plt.show()

            first_GLM = glm.first_level.FirstLevelModel(
                t_r = time_repetition,
                slice_time_ref = 0.5,
                noise_model = "ar1",
                standardize = False,
                hrf_model = "glover",
                high_pass = 1/128, 
                minimize_memory = True,
                smoothing_fwhm=6)
            
            first_fit = first_GLM.fit(
                run_imgs=func_run,
                design_matrices=design_matrix)
            
            contrast_matrix = np.eye(design_matrix.shape[1])
            contrasts = {
                column: contrast_matrix[i]
                for i, column in enumerate(design_matrix.columns)
            }
            contrasts = {
                "high": contrasts["high"],
                "low": contrasts["low"],
                "high-low": (contrasts["high"] - contrasts["low"]),
                "low-high": (contrasts["low"] - contrasts["high"])
            }

            for contrast_id, contrast_val in contrasts.items():
                constrast_map = first_fit.compute_contrast(
                    contrast_def=contrast_val,
                    output_type="all",
                    stat_type="t",
                )
                """
                plotting.plot_img_on_surf(
                    stat_map = constrast_map["stat"],
                    views=["medial"],
                    colorbar=True,
                    bg_on_data=True
                )
                """
                plotting.plot_glass_brain(
                    stat_map_img = constrast_map["z_score"],
                    threshold=scipy.stats.norm.isf(0.001),
                    title=f"z_map: {contrast_id}",
                    plot_abs=False,
                    display_mode="lyrz",
                    colorbar=True
                )
                plt.show()
                beta.append(first_fit.compute_contrast(contrast_val,output_type="effect_size").get_fdata())